# Library

In [1]:
!nvidia-smi

Wed Jun  8 10:07:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |   7471MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import cv2
import time
import random
import logging  # 로그 출력
import easydict  # 속성으로 dict 값에 access할 수 있음
import numpy as np
import pandas as pd
from tqdm import tqdm  # process bar
from os.path import join as opj
from ptflops import get_model_complexity_info
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import timm
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms

import warnings
warnings.filterwarnings('ignore')

# Config

Hyper-parameter 정의

In [3]:
args = easydict.EasyDict(
    {'exp_num':'0',
     
     # Path settings
     'data_path':'/home/lab16/jupyter_home/Data/',
     'Kfold':1,
     'model_path':'results/',

     # Model parameter settings 
     'encoder_name':'regnety_160',
     'drop_path_rate':0.2,
     
     # Training parameter settings
     ## Base Parameter
     'img_size':224,
     'batch_size':16,
     'epochs':100,
#      'optimizer':'Lamb',
     'optimizer':'Adadelta',
     'initial_lr':5e-6,
     'weight_decay':1e-3,

     ## Augmentation
     'aug_ver':2,

     ## Scheduler (OnecycleLR)
     'scheduler':'cycle',
     'warm_epoch':5,
     'max_lr':1e-3,

     ### Cosine Annealing
     'min_lr':5e-6,
     'tmax':145,

     ## etc.
     'patience':20,
     'clipping':None,

     # Hardware settings
     'amp':True,
     'multi_gpu':False,
     'logging':False,
     'num_workers':4,
     'seed':42
    })

# Utils for training and Logging

In [4]:
# Warmup Learning rate scheduler
from torch.optim.lr_scheduler import _LRScheduler
class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimizer(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

# Logging
def get_root_logger(logger_name='basicsr',
                    log_level=logging.INFO,
                    log_file=None):

    logger = logging.getLogger(logger_name)
    # if the logger has been initialized, just return it
    if logger.hasHandlers():
        return logger

    format_str = '%(asctime)s %(levelname)s: %(message)s'
    logging.basicConfig(format=format_str, level=log_level)

    if log_file is not None:
        file_handler = logging.FileHandler(log_file, 'w')
        file_handler.setFormatter(logging.Formatter(format_str))
        file_handler.setLevel(log_level)
        logger.addHandler(file_handler)

    return logger

class AvgMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.losses = []

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.losses.append(val)

# Dataset & Loader

In [5]:
# from torchvision.datasets import ImageFolder
import torch.utils.data as data
from torchvision import transforms
from glob import glob

In [6]:
import random
import math
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
# # 레이블을 one-hot-vector로 변환
# le = preprocessing.LabelEncoder()
# targets = le.fit_transform(y)
# targets = torch.as_tensor(targets)
# one_hot_y = F.one_hot(targets)

In [8]:
# validation용 추가
class Valid_Dataset(Dataset):
    def __init__(self, transform=None): 
        total_images_path = glob('/home/lab16/jupyter_home/Data/sample/*.jpg')
        file_names = []
        for i in range(len(total_images_path)):
            file_names.append(os.path.basename(total_images_path[i]))
        file_names = np.array(file_names)
        file_names.sort()

        self.file_name = file_names
        
        each_label = []
#         for i in range(len(total_images_path)):
#             each_label.append(os.path.basename(total_images_path[i])[:5])
            
        for i in range(len(total_images_path)):
            each_label.append(os.path.basename(total_images_path[i]))
            
        # 레이블을 one-hot-vector로 변환
        le = preprocessing.LabelEncoder()
        targets = le.fit_transform(each_label)
        targets = torch.as_tensor(targets)
#         one_hot_y = F.one_hot(targets)
            
        self.target = np.array(targets) # 목표는 label
#         self.target = one_hot_y
        self.transform = transform

        print(f'Validation Dataset size:{len(self.file_name)}')

    def __getitem__(self, idx):  # train 경로에 있는 png 이미지 읽어서 float32로 변환
#         image = cv2.imread(opj('./data/train_256_new/', self.file_name[idx])).astype(np.float32)
        image = cv2.imread(opj('/home/lab16/jupyter_home/Data/sample/', self.file_name[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # BGR=>RGB 변환

        target = self.target[idx]

        if self.transform is not None:
        # HWC => CHW-layout 변환
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image, target

    def __len__(self):
        return len(self.file_name)

In [9]:
class Train_Dataset(Dataset):
#     def __init__(self, df, transform=None):
    def __init__(self, transform=None):
#         self.file_name = df['file_name'].values      
        
        total_images_path = glob('/home/lab16/jupyter_home/Data/sample/*.jpg')
        file_names = []
        for i in range(len(total_images_path)):
            file_names.append(os.path.basename(total_images_path[i]))
            file_names.sort()
        file_names = np.array(file_names)

        self.file_name = file_names
                
        each_label = []
#         for i in range(len(total_images_path)):
#             each_label.append(os.path.basename(total_images_path[i])[:5])
            
        for i in range(len(total_images_path)):
            each_label.append(os.path.basename(total_images_path[i]))

        # 레이블을 one-hot-vector로 변환
        le = preprocessing.LabelEncoder()
        targets = le.fit_transform(each_label)
        targets = torch.as_tensor(targets)
#         one_hot_y = F.one_hot(targets)
            
        self.target = np.array(targets) # 목표는 label
#         self.target = one_hot_y
        self.transform = transform

        print(f'Training Dataset size:{len(self.file_name)}')
#         print(self.file_name)

    def __getitem__(self, idx):  # train 경로에 있는 png 이미지 읽어서 float32로 변환
#         image = cv2.imread(opj('./data/train_256_new/', self.file_name[idx])).astype(np.float32)
        image = cv2.imread(opj('/home/lab16/jupyter_home/Data/sample/', self.file_name[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # BGR=>RGB 변환

        target = self.target[idx]

        if self.transform is not None:
        # HWC => CHW-layout 변환
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image, target

    def __len__(self):
        return len(self.file_name)

class Test_dataset(Dataset):
    def __init__(self, path, transform=None):
#         total_images_path = glob(args.test_image_path + '*.jpg')
        total_images_path = glob(path + '*.jpg')
        file_names = []
        for i in range(len(total_images_path)):
            file_names.append(os.path.basename(total_images_path[i]))
            file_names.sort()
        file_names = np.array(file_names)

        self.test_file_name = file_names
        self.transform = transform

        print(f'Test Dataset size:{len(self.test_file_name)}')
        print(self.test_file_name)

    def __getitem__(self, idx): # test 경로에 있는 png 이미지 읽어서 float32로 변환
#         image = cv2.imread(opj('./data/test_256_new/', self.test_file_name[idx])).astype(np.float32)
#         image = cv2.imread(opj(args.test_image_path, self.test_file_name[idx])).astype(np.float32)
        image = cv2.imread(opj(path, self.test_file_name[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # BGR=>RGB 변환

        if self.transform is not None:
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image

    def __len__(self):
        return len(self.test_file_name)

# def get_loader(df, phase: str, batch_size, shuffle, num_workers, transform):
def get_loader(phase: str, batch_size, shuffle, num_workers, transform):
    if phase == 'test':
#         dataset = Test_dataset(df, transform)  
        dataset = Test_dataset(transform) 
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
        
    # 임시로 추가
    elif phase == 'validation':
#         dataset = Train_Dataset(df, transform)
        dataset = Valid_Dataset(transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
        
    else:
#         dataset = Train_Dataset(df, transform)
        path = ''
        dataset = Train_Dataset(transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True,
                                 drop_last=False)
        
    return data_loader

def get_train_augmentation(img_size, ver):
    if ver == 1: # for validset
        transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
#                 transforms.ToTensor()
                ])

    if ver == 2:
        transform = transforms.Compose([
#                 transforms.RandomHorizontalFlip(),
#                 transforms.RandomVerticalFlip(),
                transforms.RandomCrop(224),
                transforms.RandomPerspective(),
                transforms.RandomAffine((20)),  # x, y축으로 이미지 늘림
                transforms.RandomRotation(90),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                transforms.ColorJitter(brightness=0.5)
#                 transforms.ToTensor()
            ])
    
    return transform

# Network

In [10]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        # 사전 학습된 모델 사용하기
        self.encoder = timm.create_model(args.encoder_name, pretrained=True,
                                    drop_path_rate=args.drop_path_rate,
                                    )
        
        if 'regnet' in args.encoder_name:        
            num_head = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Linear(num_head, 311)
        
        elif 'efficient' in args.encoder_name:
            num_head = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Linear(num_head, 311)

    def forward(self, x):
        x = self.encoder(x)
        return x

class Network_test(nn.Module):
    def __init__(self, encoder_name):
        super().__init__()
        self.encoder = timm.create_model(encoder_name, pretrained=True,
                                    drop_path_rate=0,
                                    )
        
        if 'regnet' in encoder_name:        
            num_head = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Linear(num_head, 311)
        
        elif 'efficient' in encoder_name:
            num_head = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Linear(num_head, 311)
    
    def forward(self, x):
        x = self.encoder(x)
        return x

# Trainer for Training & Validation

In [11]:
class Trainer():
    def __init__(self, args, save_path):
        '''
        args: arguments
        save_path: Model 가중치 저장 경로
        '''
        super(Trainer, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f'device:{self.device}')

        # Logging
        log_file = os.path.join(save_path, 'log_0608_adadelta.log')
        self.logger = get_root_logger(logger_name='IR', log_level=logging.INFO, log_file=log_file)
        self.logger.info(args)
        # self.logger.info(args.tag)

        # Train, Valid Set load
        ############################################################################
#         df_train = pd.read_csv(opj(args.data_path, 'train_df.csv'))
#         print('Read train_df.csv')

#         kf = StratifiedKFold(n_splits=args.Kfold, shuffle=True, random_state=args.seed)
#         for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(df_train)), y=df_train['label'])):
#             df_train.loc[val_idx, 'fold'] = fold
#         val_idx = list(df_train[df_train['fold'] == int(args.fold)].index)

#         df_val = df_train[df_train['fold'] == args.fold].reset_index(drop=True)
#         df_train = df_train[df_train['fold'] != args.fold].reset_index(drop=True)

        # Augmentation
        self.train_transform = get_train_augmentation(img_size=args.img_size, ver=args.aug_ver)
        self.test_transform = get_train_augmentation(img_size=args.img_size, ver=1)
        
        # 수정 - TrainLoader
        self.train_loader = get_loader(phase='train', batch_size=args.batch_size, shuffle=True,
                                       num_workers=args.num_workers, transform=self.train_transform)
#         self.val_loader = get_loader(phase='validation', batch_size=args.batch_size, shuffle=False,
#                                        num_workers=args.num_workers, transform=self.train_transform)
        
        # TrainLoader
#         self.train_loader = get_loader(df_train, phase='train', batch_size=args.batch_size, shuffle=True,
#                                        num_workers=args.num_workers, transform=self.train_transform)
#         self.val_loader = get_loader(df_val, phase='train', batch_size=args.batch_size, shuffle=False,
#                                        num_workers=args.num_workers, transform=self.test_transform)

        # Network
        self.model = Network(args).to(self.device)
        macs, params = get_model_complexity_info(self.model, (3, args.img_size, args.img_size), as_strings=True,
                                                 print_per_layer_stat=False, verbose=False)
        self.logger.info('{:<30}  {:<8}'.format('Computational complexity: ', macs))
        self.logger.info('{:<30}  {:<8}'.format('Number of parameters: ', params))

        # Loss
        self.criterion = nn.CrossEntropyLoss()
        
        # Optimizer & Scheduler
#         self.optimizer = optim.Lamb(self.model.parameters(), lr=args.initial_lr, weight_decay=args.weight_decay)
        self.optimizer = torch.optim.Adadelta(self.model.parameters())
        
        iter_per_epoch = len(self.train_loader)
        self.warmup_scheduler = WarmUpLR(self.optimizer, iter_per_epoch * args.warm_epoch)

        if args.scheduler == 'step':
            self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=args.milestone, gamma=args.lr_factor, verbose=True)
        elif args.scheduler == 'cos':
            tmax = args.tmax # half-cycle 
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max = tmax, eta_min=args.min_lr, verbose=True)
        elif args.scheduler == 'cycle':
#             self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs)
            # adadelta 설정 시    
            self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs, cycle_momentum=False)

        if args.multi_gpu:
            self.model = nn.DataParallel(self.model).to(self.device)

        # Train / Validate
        best_loss = np.inf
        best_acc = 0
        best_epoch = 0
        early_stopping = 0
        start = time.time()
        for epoch in range(1, args.epochs+1):
            self.epoch = epoch

            if args.scheduler == 'cos':
                if epoch > args.warm_epoch:
                    self.scheduler.step()

            # Training
            train_loss, train_acc, train_f1 = self.training(args)

            # Model weight in Multi_GPU or Single GPU
            state_dict= self.model.module.state_dict() if args.multi_gpu else self.model.state_dict()

            # Validation
#             val_loss, val_acc, val_f1 = self.validate(args, phase='val')

            # Save models
#             if val_loss < best_loss:
#                 early_stopping = 0
#                 best_epoch = epoch
#                 best_loss = val_loss
#                 best_acc = val_acc
#                 best_f1 = val_f1

#                 torch.save({'epoch':epoch,
#                             'state_dict':state_dict,
#                             'optimizer': self.optimizer.state_dict(),
#                             'scheduler': self.scheduler.state_dict(),
#                     }, os.path.join(save_path, 'best_model_0608.pth'))
#                 self.logger.info(f'-----------------SAVE:{best_epoch}epoch----------------')
#             else:
#                 early_stopping += 1

#             # Early Stopping
#             if early_stopping == args.patience:
#                 break
                
#             print(f'\nbest epoch:{best_epoch}/loss:{best_loss:.4f}/f1:{best_f1:.4f}')
            
            # val X
            if train_loss < best_loss:
                early_stopping = 0
                best_epoch = epoch
                best_loss = train_loss
                best_acc = train_acc
                best_f1 = train_f1

                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': self.optimizer.state_dict(),
                            'scheduler': self.scheduler.state_dict(),
                    }, os.path.join(save_path, 'best_model_0608_adadelta.pth'))
                self.logger.info(f'-----------------SAVE:{best_epoch}epoch----------------')
            else:
                early_stopping += 1

            # Early Stopping
            if early_stopping == args.patience:
                break
                
            print(f'\nbest epoch:{best_epoch}/loss:{best_loss:.4f}/f1:{best_f1:.4f}')
            

#         self.logger.info(f'\nBest Val Epoch:{best_epoch} | Val Loss:{best_loss:.4f} | Val Acc:{best_acc:.4f} | Val F1:{best_f1:.4f}')
        
        self.logger.info(f'\nBest Train Epoch:{best_epoch} | Train Loss:{best_loss:.4f} | Train Acc:{best_acc:.4f} | Train F1:{best_f1:.4f}')

        end = time.time()
        self.logger.info(f'Total Process time:{(end - start) / 60:.3f}Minute')

    # Training
    def training(self, args):
        self.model.train()
        train_loss = AvgMeter()
        train_acc = 0
        preds_list = []
        targets_list = []

        scaler = grad_scaler.GradScaler()
        for i, (images, targets) in enumerate(tqdm(self.train_loader)):
            images = torch.tensor(images, device=self.device, dtype=torch.float32)
            targets = torch.tensor(targets, device=self.device, dtype=torch.long)
            
            if self.epoch <= args.warm_epoch:
                self.warmup_scheduler.step()

            self.model.zero_grad(set_to_none=True)
            if args.amp:
                with autocast():
                    preds = self.model(images)
                    loss = self.criterion(preds, targets)
                scaler.scale(loss).backward()

                # Gradient Clipping
                if args.clipping is not None:
                    scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)

                scaler.step(self.optimizer)
                scaler.update()

            else:
                preds = self.model(images)
                loss = self.criterion(preds, targets)
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)
                self.optimizer.step()

            if args.scheduler == 'cycle':
                if self.epoch > args.warm_epoch:
                    self.scheduler.step()

            # Metric
            train_acc += (preds.argmax(dim=1) == targets).sum().item()
            preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
            targets_list.extend(targets.cpu().detach().numpy())
            # log
            train_loss.update(loss.item(), n=images.size(0))

        train_acc /= len(self.train_loader.dataset)
        train_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

        self.logger.info(f'Epoch:[{self.epoch:03d}/{args.epochs:03d}]')
        self.logger.info(f'Train Loss:{train_loss.avg:.3f} | Acc:{train_acc:.4f} | F1:{train_f1:.4f}')
        return train_loss.avg, train_acc, train_f1
            
    # Validation or Dev
    def validate(self, args, phase='val'):
        self.model.eval()
        with torch.no_grad():
            val_loss = AvgMeter()
            val_acc = 0
            preds_list = []
            targets_list = []

            for i, (images, targets) in enumerate(self.val_loader):
                images = torch.tensor(images, device=self.device, dtype=torch.float32)
                targets = torch.tensor(targets, device=self.device, dtype=torch.long)

                preds = self.model(images)
                loss = self.criterion(preds, targets)

                # Metric
                val_acc += (preds.argmax(dim=1) == targets).sum().item()
                preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
                targets_list.extend(targets.cpu().detach().numpy())

                # log
                val_loss.update(loss.item(), n=images.size(0))
            val_acc /= len(self.val_loader.dataset)
            val_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

            self.logger.info(f'{phase} Loss:{val_loss.avg:.3f} | Acc:{val_acc:.4f} | F1:{val_f1:.4f}')
        return val_loss.avg, val_acc, val_f1

# Main

In [12]:
def main(args):
    print('<---- Training Params ---->')
    
    # Random Seed
    seed = args.seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

    save_path = os.path.join(args.model_path, (args.exp_num).zfill(3))
    
    # Create model directory
    os.makedirs(save_path, exist_ok=True)
    Trainer(args, save_path)

    return save_path

# Inference

In [13]:
def predict(encoder_name, test_loader, device, model_path):
    model = Network_test(encoder_name).to(device)
    model.load_state_dict(torch.load(opj(model_path, 'best_model_0608_adadelta.pth'))['state_dict'])
    model.eval()
    preds_list = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = torch.as_tensor(images, device=device, dtype=torch.float32)
            preds = model(images)
            preds = torch.softmax(preds, dim=1)
            preds_list.extend(preds.cpu().tolist())

    return np.array(preds_list)

def ensemble_5fold(model_path_list, test_loader, device):
    predict_list = []
    for model_path in model_path_list:
        prediction = predict(encoder_name= 'regnety_160', 
                             test_loader = test_loader, device = device, model_path = model_path)
        predict_list.append(prediction)
    ensemble = (predict_list[0] + predict_list[1] + predict_list[2] + predict_list[3] + predict_list[4])/len(predict_list)

    return ensemble

In [14]:
# fold X
def result(model_path_list, test_loader, device):
    predict_list = []
    for model_path in model_path_list:
        prediction = predict(encoder_name= 'regnety_160', 
                             test_loader = test_loader, device = device, model_path = model_path)
        predict_list.append(prediction)
    ensemble = predict_list[0]

    return ensemble

# Train & Inference

In [15]:
img_size = 224
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
# sub = pd.read_csv('./data/sample_submission.csv')
# df_train = pd.read_csv('./data/train_df.csv')
# df_test = pd.read_csv('./data/test_df.csv')

In [17]:
# test_transform = get_train_augmentation(img_size=img_size, ver=1)
# test_dataset = Test_dataset(df_test, test_transform)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [18]:
# fold 없이
models_path = []
args.fold = 0
args.exp_num = str(0)
save_path = main(args)
models_path.append(save_path)

2022-06-08 10:07:28,902 INFO: {'exp_num': '0', 'data_path': '/home/lab16/jupyter_home/Data/', 'Kfold': 1, 'model_path': 'results/', 'encoder_name': 'regnety_160', 'drop_path_rate': 0.2, 'img_size': 224, 'batch_size': 16, 'epochs': 100, 'optimizer': 'Adadelta', 'initial_lr': 5e-06, 'weight_decay': 0.001, 'aug_ver': 2, 'scheduler': 'cycle', 'warm_epoch': 5, 'max_lr': 0.001, 'min_lr': 5e-06, 'tmax': 145, 'patience': 20, 'clipping': None, 'amp': True, 'multi_gpu': False, 'logging': False, 'num_workers': 4, 'seed': 42, 'fold': 0}


<---- Training Params ---->
device:cuda
Training Dataset size:311


2022-06-08 10:07:30,112 INFO: Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth)
2022-06-08 10:07:34,510 INFO: Computational complexity:       15.93 GMac
2022-06-08 10:07:34,511 INFO: Number of parameters:           81.51 M 
100%|███████████████████████████████████████████| 20/20 [00:12<00:00,  1.56it/s]
2022-06-08 10:07:47,300 INFO: Epoch:[001/100]
2022-06-08 10:07:47,300 INFO: Train Loss:5.811 | Acc:0.0000 | F1:0.0000
2022-06-08 10:07:48,965 INFO: -----------------SAVE:1epoch----------------



best epoch:1/loss:5.8114/f1:0.0000


100%|███████████████████████████████████████████| 20/20 [00:08<00:00,  2.37it/s]
2022-06-08 10:07:57,423 INFO: Epoch:[002/100]
2022-06-08 10:07:57,423 INFO: Train Loss:5.754 | Acc:0.0032 | F1:0.0009
2022-06-08 10:08:01,406 INFO: -----------------SAVE:2epoch----------------


...

100%|███████████████████████████████████████████| 20/20 [00:08<00:00,  2.31it/s]
2022-06-08 10:13:30,610 INFO: Epoch:[039/100]
2022-06-08 10:13:30,611 INFO: Train Loss:5.741 | Acc:0.0000 | F1:0.0000



best epoch:21/loss:5.7283/f1:0.0000


100%|███████████████████████████████████████████| 20/20 [00:08<00:00,  2.29it/s]
2022-06-08 10:13:39,356 INFO: Epoch:[040/100]
2022-06-08 10:13:39,356 INFO: Train Loss:5.744 | Acc:0.0064 | F1:0.0010



best epoch:21/loss:5.7283/f1:0.0000


100%|███████████████████████████████████████████| 20/20 [00:08<00:00,  2.31it/s]
2022-06-08 10:13:48,010 INFO: Epoch:[041/100]
2022-06-08 10:13:48,011 INFO: Train Loss:5.740 | Acc:0.0000 | F1:0.0000
2022-06-08 10:13:48,017 INFO: 
Best Train Epoch:21 | Train Loss:5.7283 | Train Acc:0.0000 | Train F1:0.0000
2022-06-08 10:13:48,018 INFO: Total Process time:6.225Minute


In [19]:
models_path

['results/000']

In [20]:
# # fold X
# result = result(models_path, test_loader, device)

In [21]:
# 학습에 사용한 모델의 batch_size, epoch, img_size, patience
print('batch_size =', args.batch_size)
print('epochs =', args.epochs)
print('img_size =', args.img_size)
print('patience =', args.patience)

batch_size = 16
epochs = 100
img_size = 224
patience = 20


In [22]:
print('model =', args.encoder_name)

model = regnety_160
